In [1]:
from ethpandaops_python.hypersync import Hypersync
from ethpandaops_python.preprocessor import Preprocessor
import nest_asyncio
import polars as pl
import panel as pn


nest_asyncio.apply()
pn.extension("plotly", template="material", sizing_mode="stretch_width")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Data Query

In [2]:
arbitrum: str = "0xC1b634853Cb333D3aD8663715b08f41A3Aec47cc"
base: str = "0x5050F69a9786F081509234F1a7F4684b5E5b76C9"
num_days: int = 1
preprocessor: Preprocessor = Preprocessor(
    blob_producer=base,  # base
    period=num_days,
    network="mainnet",  # mainnet
)

client = Hypersync()

In [3]:
# get preprocessed data
slot_inclusion_df: pl.DataFrame = preprocessor.slot_inclusion()

txs_df = client.query_txs(address=base.lower(), period=(num_days))

In [5]:
processed_slot_inclusion_df = (
    slot_inclusion_df.drop_nulls()
    .drop(
        "meta_network_name",
        "block_root",
        "blob_size",
        "from",
        "to",
        "blob_hashes_length",
        "blob_sidecars_size",
    )
    # there is an outlier in the data that has an unusually high slot count, I think it might be an error with the xatu mempool data.
    # additionally there is currently a single tx with 5 blobs and 1 with 7. Since it's a single row, I think it's easier to just discard and ignore for now
    .filter(pl.col("slot inclusion rate") < 250)
    .filter(pl.col("slot inclusion rate (50 blob average)") < 40)
)

In [6]:
joined_processed_slot_inclusion_df = processed_slot_inclusion_df.join(
    txs_df, on="hash", how="left"
)

In [25]:
slot_inclusion_tx_df = (
    (
        joined_processed_slot_inclusion_df.with_columns(
            (pl.col("effective_gas_price") / 10**9)
            .round(3)
            .alias(
                "effective_gas_price_gwei"
            ),  # gas price in gwei that was paid, including priority fee
            (pl.col("max_fee_per_gas") / 10**9)
            .round(3)
            .alias(
                "max_fee_per_gas_gwei"
            ),  # max gas price in gwei that rollup is willing to pay
            (pl.col("max_priority_fee_per_gas") / 10**9)
            .round(3)
            # priority gas fee in gwei,
            .alias("max_priority_fee_per_gas_gwei"),
        )
    )
    .drop_nulls()
    .sort(by="submission_count", descending=True)
)

In [26]:
slot_inclusion_tx_df.head(5)

versioned_hash,nonce,event_date_time_min,event_date_time_max,fill_percentage,blob_gas,blob_gas_fee_cap,gas_price,gas_tip_cap,gas_fee_cap,hash,submission_count,slot,slot time,kzg_commitment,blob_empty_size,beacon_inclusion_time,slot inclusion rate,slot inclusion rate (50 blob average),slot target inclusion rate (2 slots),block_number,from_,to,gas,transaction_index,gas_price_right,effective_gas_price,gas_used,cumulative_gas_used,max_fee_per_gas,max_priority_fee_per_gas,effective_gas_price_gwei,max_fee_per_gas_gwei,max_priority_fee_per_gas_gwei
str,u64,datetime[ms],datetime[ms],f64,f64,f64,f64,f64,f64,str,u32,u32,datetime[ms],str,u32,f64,f64,f64,i32,i64,str,str,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0x01da7643623b4cecee8fdb9c7f047f37f25e9b5b3bb0abf2903cd6f6397c8adb""",385896,2024-04-10 16:15:11.750,2024-04-10 16:15:12.049,98.84999999999998,786432,1000000000,51292924936,2000000000,51292924936,"""0x5a899e19a9e441f6dd4f49be80bed70d2397b59f3b480057da8e5215f562be80""",1,8828476,2024-04-10 16:15:35,"""0x93a8b69adb847fede8da9312b75667d303f553b8f8af3c646cb6ab44131ca8559fe5c740d4fb837b54219a6ef7bea8b2""",0,23.25,2,1.28,2,19626356,"""0x5050f69a9786f081509234f1a7f4684b5e5b76c9""","""0xff00000000000000000000000000000000008453""",21000,62,27490942696,27490942696,21000,6966648,51292924936,2000000000,27.491,51.293,2
"""0x01808a2b7281ef3aefa6168a6847a7c12c768c53a038a730361234dfe27c0c26""",385896,2024-04-10 16:15:11.750,2024-04-10 16:15:12.049,98.84999999999998,786432,1000000000,51292924936,2000000000,51292924936,"""0x5a899e19a9e441f6dd4f49be80bed70d2397b59f3b480057da8e5215f562be80""",1,8828476,2024-04-10 16:15:35,"""0xb0a30219775b190cbb2b36b459bbc1214a8cb89e96d9bb874c1ebd4fc826c2d68fa5debb4f7c492ebf232b5d8b3d6547""",0,23.25,2,1.3,2,19626356,"""0x5050f69a9786f081509234f1a7f4684b5e5b76c9""","""0xff00000000000000000000000000000000008453""",21000,62,27490942696,27490942696,21000,6966648,51292924936,2000000000,27.491,51.293,2
"""0x0163416f790bd3cee8872fa44d223aabdd6152f6fbf49e53e41387f55fffcf83""",385896,2024-04-10 16:15:11.750,2024-04-10 16:15:12.049,98.84999999999998,786432,1000000000,51292924936,2000000000,51292924936,"""0x5a899e19a9e441f6dd4f49be80bed70d2397b59f3b480057da8e5215f562be80""",1,8828476,2024-04-10 16:15:35,"""0x8e4121723def965dffd84aedbe039c175ec04aabc22620ada32fc92a66ad597053ccf1718a30262d1c8c8964d7d837be""",0,23.25,2,1.32,2,19626356,"""0x5050f69a9786f081509234f1a7f4684b5e5b76c9""","""0xff00000000000000000000000000000000008453""",21000,62,27490942696,27490942696,21000,6966648,51292924936,2000000000,27.491,51.293,2
"""0x0183ff46baa790cc8d893eb1846736c7178e56b06c83be59386e1fe073f48298""",385896,2024-04-10 16:15:11.750,2024-04-10 16:15:12.049,98.84999999999998,786432,1000000000,51292924936,2000000000,51292924936,"""0x5a899e19a9e441f6dd4f49be80bed70d2397b59f3b480057da8e5215f562be80""",1,8828476,2024-04-10 16:15:35,"""0xa9e1d98ebe36060805bdbf71848320379b646d431b5c264d86738e1faadad5f62737696a6ed397a41db3e93cd61d81be""",0,23.25,2,1.34,2,19626356,"""0x5050f69a9786f081509234f1a7f4684b5e5b76c9""","""0xff00000000000000000000000000000000008453""",21000,62,27490942696,27490942696,21000,6966648,51292924936,2000000000,27.491,51.293,2
"""0x013b9159c3f2890c3fdf2452b104f309eea1941e6afb2499ef2b9379becb6cd5""",385896,2024-04-10 16:15:11.750,2024-04-10 16:15:12.049,98.84999999999998,786432,1000000000,51292924936,2000000000,51292924936,"""0x5a899e19a9e441f6dd4f49be80bed70d2397b59f3b480057da8e5215f562be80""",1,8828476,2024-04-10 16:15:35,"""0xb2162f0af1dc6af47382edc3b18db6e50f585e549d8e02d3554eb7d596915c386f342fa24b11386a79fba54b098289f0""",9032,23.25,2,1.36,2,19626356,"""0x5050f69a9786f081509234f1a7f4684b5e5b76c9""","""0xff00000000000000000000000000000000008453""",21000,62,27490942696,27490942696,21000,6966648,51292924936,2000000000,27.491,51.293,2


### Slot Inclusion Time Charts

In [27]:
slot_inclusion_line_chart = (
    slot_inclusion_tx_df.drop_nulls()
    .select(
        "slot time",
        "slot inclusion rate",
        "slot inclusion rate (50 blob average)",
        "slot target inclusion rate (2 slots)",
    )
    .plot.line(
        x="slot time",
        y=[
            "slot inclusion rate",
            "slot inclusion rate (50 blob average)",
            "slot target inclusion rate (2 slots)",
        ],
        color=["blue", "red", "black"],
        ylabel="Beacon Block Inclusion (block)",
        xlabel="Slot Date Time",
        title="Beacon Slot Inclusion Time",
        # TODO - color by resubmission count
        width=1000,
        height=400,
    )
)

In [28]:
slot_inclusion_histogram_chart = (
    slot_inclusion_tx_df.with_columns(
        pl.when(pl.col("slot inclusion rate") > 5)
        .then(5)
        .otherwise(pl.col("slot inclusion rate"))
        .alias("slot inclusion rate > 5"),
    )
).plot.hist(
    "slot inclusion rate > 5",
    ylabel="blob count",
    xlabel="slot inclusion rate",
    title="Slot Inclusion Distribution (slot inclusion rate > 5 grouped together)",
    bins=40,
    width=600,
    height=400,
)

### Slot Inclusion Gas Cost Variance

In [39]:
priority_fee_premium_df = slot_inclusion_tx_df.with_columns(
    (
        (pl.col("max_priority_fee_per_gas_gwei") / pl.col("effective_gas_price_gwei"))
        * 100
    )
    .round(3)
    .alias("priority_fee_bid_percent_premium")
)

In [65]:
gas_fluctuations_chart = (
    (
        priority_fee_premium_df.select(
            "block_number",
            "max_priority_fee_per_gas_gwei",
            "effective_gas_price_gwei",
            "priority_fee_bid_percent_premium",
            "slot inclusion rate",
        )
        .unique()
        .sort(by="block_number")
        .with_columns(
            (
                # estimate min block gas by taking the gwei paid minus the priority fee
                pl.col("effective_gas_price_gwei")
                - pl.col("max_priority_fee_per_gas_gwei")
            ).alias("min_block_gas_gwei")
        )
        .with_columns(
            # calculate per tx gas fluctuations
            pl.col("min_block_gas_gwei").diff().abs().alias(
                "gas_fluctuation_gwei")
        )
        .with_columns(
            (pl.col("gas_fluctuation_gwei") / pl.col("min_block_gas_gwei") * 100).alias(
                "gas_fluctuation_percent"
            )
        )
    )
    .group_by("slot inclusion rate")
    .agg(
        pl.col("gas_fluctuation_percent").mean(),
        pl.col("priority_fee_bid_percent_premium").mean(),
    )
    .sort(by="slot inclusion rate")
    .plot.bar(
        x="slot inclusion rate",
        y="gas_fluctuation_percent",
        ylabel="gas fluctuation %",
        title="gas fluctuation percent over time",
        width=800,
        height=400,
    )
)

priority_fee_premium_chart = (
    (
        priority_fee_premium_df.select(
            "block_number",
            "max_priority_fee_per_gas_gwei",
            "effective_gas_price_gwei",
            "priority_fee_bid_percent_premium",
            "slot inclusion rate",
        )
        .unique()
        .sort(by="block_number")
        .with_columns(
            (
                # estimate min block gas by taking the gwei paid minus the priority fee
                pl.col("effective_gas_price_gwei")
                - pl.col("max_priority_fee_per_gas_gwei")
            ).alias("min_block_gas_gwei")
        )
        .with_columns(
            # calculate per tx gas fluctuations
            pl.col("min_block_gas_gwei").diff().abs().alias(
                "gas_fluctuation_gwei")
        )
        .with_columns(
            (pl.col("gas_fluctuation_gwei") / pl.col("min_block_gas_gwei") * 100).alias(
                "gas_fluctuation_percent"
            )
        )
    )
    .group_by("slot inclusion rate")
    .agg(
        pl.col("gas_fluctuation_percent").mean(),
        pl.col("priority_fee_bid_percent_premium").mean(),
    )
    .sort(by="slot inclusion rate")
    .plot.bar(
        x="slot inclusion rate",
        y="priority_fee_bid_percent_premium",
        ylabel="priority fee bid premium %",
        title="priority fee bid premium over time",
        width=800,
        height=400,
    )
)

### Assemble Dashboard

In [68]:
dash = pn.Column(
    pn.pane.Markdown(
        """
        # Blob Inclusion Dashboard
        When a transaction is resubmitted with updated gas parameters, the transaction hash changes. 
        For example take this blob reference hash - 0x01c738cf37c911334c771f2295c060e5bd7d084f347e4334863336724934c59a. On [etherscan](https://etherscan.io/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69) 
        we can see that the transaction was replaced by the user. We can see on Ethernow that the transaction contains the same 
        blob reference hash in both the [original tx](https://www.ethernow.xyz/tx/0x763d823c0f933c4d2eb84406b37aa2649753f2f563fa3ee6d27251c6a52a8d69?batchIndex=1) and the [resubmitted tx](https://www.ethernow.xyz/tx/0x5a4094662bd05ff3639a8979927ab527e007a6925387951a9c1b3d2958b13a86?batchIndex=1).
        
        We can measure the total time that a blob hash sat in the mempool by subtracting the original tx was first seen from the slot time, when it eventually is finalized on the beacon chain. 
        In this particular example, the total time that the blob sat in the mempool was not from 18:56:27 to 18:57:11 (4 slots), but really 18:54:29 to 18:57:11 (14 slots)
        """
    ),
    pn.Column(
        pn.pane.Markdown(
            """
        ## Slot Inclusion Time Charts
        This time-series chart to the left shows the slot inclusion performance over time from when it's first seen in the mempool to when it gets finalized in the beacon block.
        A 50 blob slot inclusion average is taken to smooth out the performance. The target slot inclusion rate is 2. To the right is the overall distribution of the slot inclusion rate.
        While a lot of blobs get finalized on the beacon chain within 1-2 blocks, there are still a non-trivial amount of outliers that can take upwards of 5-10 blocks before finalizing.
        """
        ),
        pn.Row(slot_inclusion_line_chart, slot_inclusion_histogram_chart),
        pn.pane.Markdown(
            """
        ## Slot Inclusion Gas Cost Variance
        These histogram charts show the distribution of gas fluctuations and priority fee bid premiums over time over the slot inclusion rate period. The left histogram shows the gas variance over time.
        The higher the gas variance over the slot inclusion period, the more likely it is that the gas parameters will be out of date and need to be resubmitted. The right histogram shows the 
        priority fee bid premium over time. There isn't a strong correlation between higher priority fee bid premiums and faster slot inclusion rates. However the longer the slot inclusion rate is, the
        more likely that the priority fee bid that is submitted will be inefficient with respect to the current gas market conditions. 
        """
        ),
        pn.Row(gas_fluctuations_chart, priority_fee_premium_chart),
    ),
)

In [69]:
# dash.show()
dash.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'e60749a5-fab9-47bd-8a56-cc49bf281917': {'version…